In [187]:
import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib.pyplot as plt
import sys
import os
import requests
from jinja2 import Environment, FileSystemLoader
import logging


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [188]:
#Set state you want to run
state='New Mexico'

In [189]:
#import state info and get state abbreviation and fip code
state_info = pd.read_csv('./State_Info.csv')

abbrv = state_info.loc[state_info['State'] == state, 'Abbreviation'].iloc[0]
fip = str(state_info.loc[state_info['State'] == state, 'FIPS'].iloc[0])
fip = fip.zfill(2)

In [190]:
#import place and county csvs
df_2020_place = pd.read_csv('./place_csv/' + abbrv + '_pl2020_place.csv',dtype={'GEOCODE':str})
df_2020_cnty = pd.read_csv('./cnty_csv/' + abbrv + '_pl2020_cnty.csv',dtype={'GEOCODE':str})

In [193]:
#filter dataframes to only include these columns
col_list_place = ['NAME','P0010001','STATE','PLACE','GEOCODE']
df_2020_place = df_2020_place[col_list_place]
col_list_cnty = col_list_place
col_list_cnty[-2] = 'COUNTY' 
df_2020_cnty =df_2020_cnty[col_list_cnty]

In [194]:
#convert to strings
df_2020_place = df_2020_place.applymap(str)
df_2020_cnty = df_2020_cnty.applymap(str)
df_2020_cnty.head(10)

,NAME,P0010001,STATE,COUNTY,GEOCODE
0,Bernalillo County,676444,35,1.0,35001
1,Catron County,3579,35,3.0,35003
2,Chaves County,65157,35,5.0,35005
3,Cibola County,27172,35,6.0,35006
4,Colfax County,12387,35,7.0,35007
5,Curry County,48430,35,9.0,35009
6,De Baca County,1698,35,11.0,35011
7,Doña Ana County,219561,35,13.0,35013
8,Eddy County,62314,35,15.0,35015
9,Grant County,28185,35,17.0,35017


In [195]:
#Get total pop from adding county populations
total_pop = pd.to_numeric(df_2020_cnty['P0010001'])
total_pop = total_pop.sum()
total_pop

2117522

In [196]:
#Get resident population from state info
state_pop = pd.to_numeric(state_info.loc[state_info['State'] == state, 'Resident Pop'].iloc[0])
state_pop

2117522

In [197]:
#compare populations to make sure they match
if total_pop != state_pop:
    print('POPULATIONS NOT EQUAL!!!')
    sys.exit("POPULATIONS NOT EQUAL!!!")

In [198]:
#What variables to aquire from census API
variables = ['NAME', 'P001001']
variables

['NAME', 'P001001']

In [199]:
#Function to get census API data
def get_2010pl_data(fip, geog, variables = variables, CENSUS_API_KEY = '2eeb19ba591457182b8b757f5c860cba21bbe7ec'):
    HOST = "https://api.census.gov/data"
    year = "2010"
    dataset = "dec/pl"
    base_url = "/".join([HOST, year, dataset])
    print('starting to collect data for ' + geog + ' ' + fip)
    predicates = {} 
    predicates["get"] = ",".join(variables)
    if geog == 'place':
        predicates["for"] = "place:*"
    if geog == 'cnty':
        predicates["for"] = "county:*"
    predicates["in"] = "state:" + fip
    predicates["key"] = CENSUS_API_KEY
    # Write the result to a response object
    response = requests.get(base_url, params=predicates)
    col_names = response.json()[0]        
    data = response.json()[1:]
    print('done collecting data for', fip)
    geoids = []  # initialize geoid vector
    pop_data = pd.DataFrame(columns=col_names, data=data)
    cols = [i for i in pop_data.columns if i not in ["NAME","place","state","county"]]
    for col in cols:
        pop_data[col]=pd.to_numeric(pop_data[col])
    for index, row in pop_data.iterrows():
        # make changes here for tracts
        if geog == 'place':
            geoid = row["state"] + row["place"]
        if geog == 'cnty':
            geoid = row["state"] + row["county"]
        geoids.append(geoid)
    pop_data["GEOID"] = geoids
    return pop_data

In [200]:
#df_2010_pl = get_2010pl_data('56','place')
#df_2010_pl = df_2010_pl[~df_2010_pl.NAME.str.contains(" CDP")]
#df_2010_pl.shape

In [201]:
#Call API and get places
df_2010_place = get_2010pl_data(fip,'place')
df_2010_place.columns = col_list_place
df_2010_place = df_2010_place.applymap(str)
df_2010_place.head(10)

starting to collect data for place 35
done collecting data for 35


,NAME,P0010001,STATE,COUNTY,GEOCODE
0,"Abeytas CDP, New Mexico",56,35,00240,3500240
1,"Abiquiu CDP, New Mexico",231,35,00310,3500310
2,"Acomita Lake CDP, New Mexico",416,35,00765,3500765
3,"Adelino CDP, New Mexico",823,35,00940,3500940
4,"Agua Fria CDP, New Mexico",2800,35,01220,3501220
5,"Alamillo CDP, New Mexico",102,35,01640,3501640
6,"Alamo CDP, New Mexico",1085,35,01710,3501710
7,"Alamogordo city, New Mexico",30403,35,01780,3501780
8,"Albuquerque city, New Mexico",545852,35,02000,3502000
9,"Alcalde CDP, New Mexico",285,35,02070,3502070


In [202]:
#Call API and get counties
df_2010_cnty = get_2010pl_data(fip,'cnty')
df_2010_cnty.columns = col_list_cnty
df_2010_cnty = df_2010_cnty.applymap(str)
df_2010_cnty.head(10)

starting to collect data for cnty 35
done collecting data for 35


,NAME,P0010001,STATE,COUNTY,GEOCODE
0,"Bernalillo County, New Mexico",662564,35,001,35001
1,"Catron County, New Mexico",3725,35,003,35003
2,"Chaves County, New Mexico",65645,35,005,35005
3,"Cibola County, New Mexico",27213,35,006,35006
4,"Colfax County, New Mexico",13750,35,007,35007
5,"Curry County, New Mexico",48376,35,009,35009
6,"De Baca County, New Mexico",2022,35,011,35011
7,"Dona Ana County, New Mexico Doña Ana County",209233,35,013,35013
8,"Eddy County, New Mexico",53829,35,015,35015
9,"Grant County, New Mexico",29514,35,017,35017


In [203]:
#Join place dataframes
join_attempt_place = pd.merge(df_2010_place,df_2020_place,how="outer",on="GEOCODE",indicator=True)
print(join_attempt_place["_merge"].value_counts())

both          438
right_only     89
left_only       5
Name: _merge, dtype: int64


In [204]:
#Check non-joined rows
join_attempt_place[join_attempt_place['_merge'] != 'both'][['NAME_x','NAME_y','GEOCODE']]

,NAME_x,NAME_y,GEOCODE
35,"Bluewater Acres CDP, New Mexico",NaN,3508185
50,"Gallina CDP, New Mexico",NaN,3512008
165,"Isleta Village Proper CDP, New Mexico",NaN,3534760
302,"Pueblo CDP, New Mexico",NaN,3559860
439,"Young Place CDP, New Mexico",NaN,3586175
443,NaN,Alma CDP,3502560
444,NaN,Angustura CDP,3503680
445,NaN,Arroyo Hondo CDP,3505002
446,NaN,Barton CDP,3506220
447,NaN,Becenti CDP,3506340


In [205]:
if join_attempt_place[join_attempt_place['_merge'] != 'both'][['NAME_x','NAME_y','GEOCODE']].shape[0] != 0:
    print('Places DO NOT MATCH!!!')
    #sys.exit("COUNTIES DO NOT MATCH!!!")

Places DO NOT MATCH!!!


In [206]:
place_join = join_attempt_place[join_attempt_place["_merge"]=="both"]

In [207]:
place_join.columns = place_join.columns.str.replace("_x", "_2010")
place_join.columns = place_join.columns.str.replace("_y", "_2020")

In [208]:
join_attempt_cnty = pd.merge(df_2010_cnty,df_2020_cnty,how="outer",on="GEOCODE",indicator=True)
print(join_attempt_cnty["_merge"].value_counts())

both          33
left_only      0
right_only     0
Name: _merge, dtype: int64


In [209]:
#Check non-joined rows
join_attempt_cnty[join_attempt_cnty['_merge'] != 'both'][['NAME_x','NAME_y','GEOCODE']]

,NAME_x,NAME_y,GEOCODE


In [210]:
if join_attempt_cnty[join_attempt_cnty['_merge'] != 'both'][['NAME_x','NAME_y','GEOCODE']].shape[0] != 0:
    print('COUNTIES DO NOT MATCH!!!')
    sys.exit("COUNTIES DO NOT MATCH!!!")

In [211]:
cnty_join = join_attempt_cnty[join_attempt_cnty["_merge"]=="both"]

In [212]:
#print(cnty_join.shape)
#cnty_join = cnty_join[cnty_join.NAME_y != 'Bedford County']
#cnty_join.shape

In [213]:
cnty_join.columns = cnty_join.columns.str.replace("_x", "_2010")
cnty_join.columns = cnty_join.columns.str.replace("_y", "_2020")

In [214]:
#Drop columns and rename remaining
place_join = place_join.drop(columns=['NAME_2010', 'COUNTY','STATE_2010','_merge'])
place_join = place_join.rename(columns={'NAME_2020': 'NAME', 'STATE_2020': 'STATE'})
place_join = place_join.reindex(sorted(place_join.columns), axis=1)
place_join.head(10)

,GEOCODE,NAME,P0010001_2010,P0010001_2020,PLACE,STATE
0,3500240,Abeytas CDP,56,45,240.0,35
1,3500310,Abiquiu CDP,231,181,310.0,35
2,3500765,Acomita Lake CDP,416,339,765.0,35
3,3500940,Adelino CDP,823,735,940.0,35
4,3501220,Agua Fria CDP,2800,2913,1220.0,35
5,3501640,Alamillo CDP,102,124,1640.0,35
6,3501710,Alamo CDP,1085,1150,1710.0,35
7,3501780,Alamogordo city,30403,30898,1780.0,35
8,3502000,Albuquerque city,545852,564559,2000.0,35
9,3502070,Alcalde CDP,285,310,2070.0,35


In [215]:
cnty_join = cnty_join.drop(columns=['NAME_2010', 'COUNTY_2020','STATE_2010','_merge'])
cnty_join = cnty_join.rename(columns={'NAME_2020': 'NAME', 'STATE_2020': 'STATE', 'COUNTY_2010': 'COUNTY'})
cnty_join = cnty_join.reindex(sorted(cnty_join.columns), axis=1)
cnty_join.head(10)

,COUNTY,GEOCODE,NAME,P0010001_2010,P0010001_2020,STATE
0,001,35001,Bernalillo County,662564,676444,35
1,003,35003,Catron County,3725,3579,35
2,005,35005,Chaves County,65645,65157,35
3,006,35006,Cibola County,27213,27172,35
4,007,35007,Colfax County,13750,12387,35
5,009,35009,Curry County,48376,48430,35
6,011,35011,De Baca County,2022,1698,35
7,013,35013,Doña Ana County,209233,219561,35
8,015,35015,Eddy County,53829,62314,35
9,017,35017,Grant County,29514,28185,35


In [216]:
#Write Excel with Pop changes
with pd.ExcelWriter('./ExcelFiles/'+state+'_PopChanges.xlsx') as writer:place_join.to_excel(writer, sheet_name='Census Place Population Changes'),cnty_join.to_excel(writer, sheet_name='County Population Changes')

In [217]:
if state != 'Hawaii':
    place_join = place_join[~place_join.NAME.str.contains(" CDP")]

In [218]:
#Rename columns for tables
place_join = place_join[['NAME','P0010001_2010','P0010001_2020']]
if state == 'Hawaii':
    place_join.columns = ['Census Place Name', 'Population 2010', 'Population 2020']
    
elif state != 'Hawaii':
    place_join.columns = ['Municipality Name', 'Population 2010', 'Population 2020']

In [219]:
#convert to numeric
place_join[['Population 2010', 'Population 2020']] = place_join[['Population 2010', 'Population 2020']].apply(pd.to_numeric)

In [220]:
#Get min pop for the places in state
num_pop = total_pop * .0005
myList1 = [500,1000,5000,10000]
num1_pop = min(myList1, key=lambda x:abs(x-num_pop))
num1_pop

1000

In [221]:
#Remove all places less than X people
place_join = place_join[(place_join["Population 2010"]>= num1_pop) & (place_join["Population 2020"]>=num1_pop)]
#Find difference in 2010 and 2020 pop
place_join['Population Change']=place_join['Population 2020']-place_join['Population 2010']
#Find # difference in 2010 and 2020
place_join['% Dif']=((place_join['Population Change']/place_join['Population 2010'])*100)
place_join['Percent Change']=((place_join['Population Change']/place_join['Population 2010'])*100).round(2).astype(str) + '%'



In [222]:
#Get top 20% of places
num = place_join.shape[0] * .2
myList = [3,5,10,20]

In [223]:
num1 = min(myList, key=lambda x:abs(x-num))

In [224]:
#Create place tables
place_join = place_join.sort_values(by=['% Dif'], ascending=False)
Census_Pop_Growth = place_join.head(num1)

place_join = place_join.sort_values(by=['% Dif'], ascending=True)
Census_Pop_Loss = place_join.head(num1)

place_join = place_join.sort_values(by=['Population Change'], ascending=False)
Census_Pop_Largest_Growth = place_join.head(num1)

In [225]:
del Census_Pop_Growth['% Dif']
#Add commas
Census_Pop_Growth['Population 2010'] = Census_Pop_Growth['Population 2010'].apply(lambda x : "{:,}".format(x))
Census_Pop_Growth['Population 2020'] = Census_Pop_Growth['Population 2020'].apply(lambda x : "{:,}".format(x))
Census_Pop_Growth['Population Change'] = Census_Pop_Growth['Population Change'].apply(lambda x : "{:,}".format(x))
Census_Pop_Growth.index = np.arange(1, len(Census_Pop_Growth) + 1)
Census_Pop_Growth

,Municipality Name,Population 2010,Population 2020,Population Change,Percent Change
1,Edgewood town,"3,735","6,174","2,439",65.3%
2,Santa Fe city,"67,947","87,505","19,558",28.78%
3,Carlsbad city,"26,138","32,238","6,100",23.34%
4,Rio Rancho city,"87,521","104,046","16,525",18.88%
5,Hobbs city,"34,122","40,508","6,386",18.72%
6,Sunland Park city,"14,106","16,702","2,596",18.4%
7,Los Lunas village,"14,835","17,242","2,407",16.23%
8,Las Cruces city,"97,618","111,385","13,767",14.1%
9,Artesia city,"11,301","12,875","1,574",13.93%
10,Taos town,"5,716","6,474",758,13.26%


In [226]:
del Census_Pop_Loss['% Dif']
#Add commas
Census_Pop_Loss['Population 2010'] = Census_Pop_Loss['Population 2010'].apply(lambda x : "{:,}".format(x))
Census_Pop_Loss['Population 2020'] = Census_Pop_Loss['Population 2020'].apply(lambda x : "{:,}".format(x))
Census_Pop_Loss['Population Change'] = Census_Pop_Loss['Population Change'].apply(lambda x : "{:,}".format(x))
Census_Pop_Loss.index = np.arange(1, len(Census_Pop_Loss) + 1)
Census_Pop_Loss

,Municipality Name,Population 2010,Population 2020,Population Change,Percent Change
1,Estancia town,"1,655","1,242",-413,-24.95%
2,Milan village,"3,245","2,456",-789,-24.31%
3,Mesilla town,"2,196","1,797",-399,-18.17%
4,Lordsburg city,"2,797","2,335",-462,-16.52%
5,Dexter town,"1,266","1,074",-192,-15.17%
6,Columbus village,"1,664","1,442",-222,-13.34%
7,Raton city,"6,885","6,041",-844,-12.26%
8,Clayton town,"2,980","2,643",-337,-11.31%
9,Tularosa village,"2,842","2,553",-289,-10.17%
10,Bayard city,"2,328","2,116",-212,-9.11%


In [227]:
del Census_Pop_Largest_Growth['% Dif']
#Add commas
Census_Pop_Largest_Growth['Population 2010'] = Census_Pop_Largest_Growth['Population 2010'].apply(lambda x : "{:,}".format(x))
Census_Pop_Largest_Growth['Population 2020'] = Census_Pop_Largest_Growth['Population 2020'].apply(lambda x : "{:,}".format(x))
Census_Pop_Largest_Growth['Population Change'] = Census_Pop_Largest_Growth['Population Change'].apply(lambda x : "{:,}".format(x))
Census_Pop_Largest_Growth.index = np.arange(1, len(Census_Pop_Largest_Growth) + 1)
Census_Pop_Largest_Growth

<ipython-input-227-b1958ec3cc11>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Census_Pop_Largest_Growth['Population 2010'] = Census_Pop_Largest_Growth['Population 2010'].apply(lambda x : "{:,}".format(x))
<ipython-input-227-b1958ec3cc11>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Census_Pop_Largest_Growth['Population 2020'] = Census_Pop_Largest_Growth['Population 2020'].apply(lambda x : "{:,}".format(x))
<ipython-input-227-b1958ec3cc11>:5: SettingWithCopyWarning: 
A value is trying to be set on a 

,Municipality Name,Population 2010,Population 2020,Population Change,Percent Change
1,Santa Fe city,"67,947","87,505","19,558",28.78%
2,Albuquerque city,"545,852","564,559","18,707",3.43%
3,Rio Rancho city,"87,521","104,046","16,525",18.88%
4,Las Cruces city,"97,618","111,385","13,767",14.1%
5,Hobbs city,"34,122","40,508","6,386",18.72%
6,Carlsbad city,"26,138","32,238","6,100",23.34%
7,Sunland Park city,"14,106","16,702","2,596",18.4%
8,Edgewood town,"3,735","6,174","2,439",65.3%
9,Los Lunas village,"14,835","17,242","2,407",16.23%
10,Artesia city,"11,301","12,875","1,574",13.93%


In [228]:
cnty_join = cnty_join[['NAME','P0010001_2010','P0010001_2020']]
cnty_join.columns = ['County', 'Population 2010', 'Population 2020']

In [229]:
cnty_join[['Population 2010', 'Population 2020']] = cnty_join[['Population 2010', 'Population 2020']].apply(pd.to_numeric)

In [230]:
#Find difference in 2010 and 2020 pop
cnty_join['Population Change']=cnty_join['Population 2020']-cnty_join['Population 2010']
#Find # difference in 2010 and 2020
cnty_join['% Dif']=((cnty_join['Population Change']/cnty_join['Population 2010'])*100)
cnty_join['Percent Change']=((cnty_join['Population Change']/cnty_join['Population 2010'])*100).round(2).astype(str) + '%'

In [231]:
#Create county tables
if cnty_join.shape[0] <= 20:
    cnty_join = cnty_join.sort_values(by=['% Dif'], ascending=False)
    County_Pop_Growth = cnty_join
    
elif cnty_join.shape[0] > 20:
    cnty_join = cnty_join.sort_values(by=['% Dif'], ascending=False)
    County_Pop_Growth = cnty_join.head(10)
    
    
cnty_join = cnty_join.sort_values(by=['% Dif'], ascending=True)
County_Pop_Loss = cnty_join.head(10)

cnty_join = cnty_join.sort_values(by=['Population Change','% Dif'], ascending=False)
County_Pop_Largest_Growth = cnty_join.head(10)

In [232]:
del County_Pop_Growth['% Dif']
#Add commas
County_Pop_Growth['Population 2010'] = County_Pop_Growth['Population 2010'].apply(lambda x : "{:,}".format(x))
County_Pop_Growth['Population 2020'] = County_Pop_Growth['Population 2020'].apply(lambda x : "{:,}".format(x))
County_Pop_Growth['Population Change'] = County_Pop_Growth['Population Change'].apply(lambda x : "{:,}".format(x))
County_Pop_Growth.index = np.arange(1, len(County_Pop_Growth) + 1)
County_Pop_Growth

,County,Population 2010,Population 2020,Population Change,Percent Change
1,Eddy County,"53,829","62,314","8,485",15.76%
2,Lea County,"64,727","74,455","9,728",15.03%
3,Sandoval County,"131,561","148,834","17,273",13.13%
4,Los Alamos County,"17,950","19,419","1,469",8.18%
5,Santa Fe County,"144,170","154,823","10,653",7.39%
6,Otero County,"63,797","67,839","4,042",6.34%
7,Doña Ana County,"209,233","219,561","10,328",4.94%
8,Taos County,"32,937","34,489","1,552",4.71%
9,Bernalillo County,"662,564","676,444","13,880",2.09%
10,McKinley County,"71,492","72,902","1,410",1.97%


In [233]:
del County_Pop_Loss['% Dif']
#Add commas
County_Pop_Loss['Population 2010'] = County_Pop_Loss['Population 2010'].apply(lambda x : "{:,}".format(x))
County_Pop_Loss['Population 2020'] = County_Pop_Loss['Population 2020'].apply(lambda x : "{:,}".format(x))
County_Pop_Loss['Population Change'] = County_Pop_Loss['Population Change'].apply(lambda x : "{:,}".format(x))
County_Pop_Loss.index = np.arange(1, len(County_Pop_Loss) + 1)
County_Pop_Loss

,County,Population 2010,Population 2020,Population Change,Percent Change
1,De Baca County,"2,022","1,698",-324,-16.02%
2,Hidalgo County,"4,894","4,178",-716,-14.63%
3,Mora County,"4,881","4,189",-692,-14.18%
4,Union County,"4,549","4,079",-470,-10.33%
5,Colfax County,"13,750","12,387","-1,363",-9.91%
6,Torrance County,"16,383","15,045","-1,338",-8.17%
7,San Miguel County,"29,393","27,201","-2,192",-7.46%
8,Socorro County,"17,866","16,595","-1,271",-7.11%
9,San Juan County,"130,044","121,661","-8,383",-6.45%
10,Harding County,695,657,-38,-5.47%


In [234]:
del County_Pop_Largest_Growth['% Dif']
#Add commas
County_Pop_Largest_Growth['Population 2010'] = County_Pop_Largest_Growth['Population 2010'].apply(lambda x : "{:,}".format(x))
County_Pop_Largest_Growth['Population 2020'] = County_Pop_Largest_Growth['Population 2020'].apply(lambda x : "{:,}".format(x))
County_Pop_Largest_Growth['Population Change'] = County_Pop_Largest_Growth['Population Change'].apply(lambda x : "{:,}".format(x))
County_Pop_Largest_Growth.index = np.arange(1, len(County_Pop_Largest_Growth) + 1)
County_Pop_Largest_Growth

<ipython-input-234-95f6d673028e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  County_Pop_Largest_Growth['Population 2010'] = County_Pop_Largest_Growth['Population 2010'].apply(lambda x : "{:,}".format(x))
<ipython-input-234-95f6d673028e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  County_Pop_Largest_Growth['Population 2020'] = County_Pop_Largest_Growth['Population 2020'].apply(lambda x : "{:,}".format(x))
<ipython-input-234-95f6d673028e>:5: SettingWithCopyWarning: 
A value is trying to be set on a 

,County,Population 2010,Population 2020,Population Change,Percent Change
1,Sandoval County,"131,561","148,834","17,273",13.13%
2,Bernalillo County,"662,564","676,444","13,880",2.09%
3,Santa Fe County,"144,170","154,823","10,653",7.39%
4,Doña Ana County,"209,233","219,561","10,328",4.94%
5,Lea County,"64,727","74,455","9,728",15.03%
6,Eddy County,"53,829","62,314","8,485",15.76%
7,Otero County,"63,797","67,839","4,042",6.34%
8,Taos County,"32,937","34,489","1,552",4.71%
9,Los Alamos County,"17,950","19,419","1,469",8.18%
10,McKinley County,"71,492","72,902","1,410",1.97%


In [235]:
cnty_join.shape[0]

33

In [236]:
#Choose template to use
env = Environment(loader=FileSystemLoader('.'))


if cnty_join.shape[0] <= 20:
    template = env.get_template("Report2.htm")
    
elif cnty_join.shape[0] > 20:
    template = env.get_template("Report1.htm")

if state == 'Hawaii':
    template = env.get_template("Report3.htm")
template

<Template 'Report1.htm'>

In [237]:
num1_pop = format(num1_pop, ",")
num1_pop

'1,000'

In [238]:
#Add variables and tables
template_vars = {"State":state,
                'Num_Places':num1,
                'Table1':Census_Pop_Growth.to_html(index=True),
                'Table2':Census_Pop_Loss.to_html(index=True),
                'Table3':Census_Pop_Largest_Growth.to_html(index=True),
                'Table4':County_Pop_Growth.to_html(index=True),
                'Table5':County_Pop_Loss.to_html(index=True),
                'Table6':County_Pop_Largest_Growth.to_html(index=True),
                'Min_Pop':num1_pop
                
                }

In [239]:
#Save as html
html_out = template.render(template_vars)
Html_file= open("./Reports/"+ state + "_Population_Changes.html","w", encoding="utf-8")
Html_file.write(html_out)
Html_file.close()